In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

def plot_spectrogram(audio_path, output_image_path):
    y, sr = librosa.load(audio_path, sr=None)

    # Построение спектрограммы с использованием окна Ханна
    n_fft = 2048
    hop_length = 512
    window = 'hann'

    # Вычисляем STFT
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window))

    # Преобразование амплитуд в дБ
    DB = librosa.amplitude_to_db(D, ref=np.max)

    # Визуализация спектрограммы
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram (with Hann window)')
    plt.savefig(output_image_path)
    plt.show()

# Примеры использования:
plot_spectrogram('a.wav', 'a_spectrogram.png')
plot_spectrogram('i.wav', 'i_spectrogram.png')
plot_spectrogram('bark.wav', 'bark_spectrogram.png')

In [ ]:
def find_min_max_frequency(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    # Построение спектрограммы
    n_fft = 2048
    hop_length = 512
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    # Определение минимальной и максимальной частоты
    nonzero_freqs = freqs[np.max(DB, axis=1) > -80]  # Учитываем частоты с амплитудой выше порога
    min_freq = np.min(nonzero_freqs)
    max_freq = np.max(nonzero_freqs)
    
    return min_freq, max_freq

# Примеры использования:
min_freq_a, max_freq_a = find_min_max_frequency('a.wav')
min_freq_i, max_freq_i = find_min_max_frequency('i.wav')
min_freq_bark, max_freq_bark = find_min_max_frequency('bark.wav')

print(f"Min and Max Frequency for 'A': {min_freq_a} Hz, {max_freq_a} Hz")
print(f"Min and Max Frequency for 'I': {min_freq_i} Hz, {max_freq_i} Hz")
print(f"Min and Max Frequency for Bark: {min_freq_bark} Hz, {max_freq_bark} Hz")


In [ ]:
def find_most_tonal_frequency(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    # Построение спектрограммы
    n_fft = 2048
    hop_length = 512
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    # Поиск частоты с наибольшим количеством обертонов
    harmonic_freqs = librosa.yin(y, fmin=50, fmax=2000, sr=sr, hop_length=hop_length)
    harmonic_counts = np.histogram(harmonic_freqs, bins=np.arange(0, sr // 2, 50))[0]
    most_tonal_frequency = np.argmax(harmonic_counts) * 50
    
    return most_tonal_frequency

# Примеры использования:
tonal_freq_a = find_most_tonal_frequency('a.wav')
tonal_freq_i = find_most_tonal_frequency('i.wav')
tonal_freq_bark = find_most_tonal_frequency('bark.wav')

print(f"Most Tonal Frequency for 'A': {tonal_freq_a} Hz")
print(f"Most Tonal Frequency for 'I': {tonal_freq_i} Hz")
print(f"Most Tonal Frequency for Bark: {tonal_freq_bark} Hz")


In [ ]:
def find_formants(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    n_fft = 2048
    hop_length = 512
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    DB = librosa.amplitude_to_db(D, ref=np.max)

    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    time_steps = librosa.frames_to_time(np.arange(D.shape[1]), sr=sr, hop_length=hop_length)
    
    # Определение формантов
    formants = []
    for t in range(0, len(time_steps), int(0.1 * sr / hop_length)):
        spectrum_slice = DB[:, t]
        peak_indices = np.argpartition(spectrum_slice, -3)[-3:]  # Три самых сильных пика
        formant_frequencies = freqs[peak_indices]
        formants.append(formant_frequencies)
    
    return formants

# Примеры использования:
formants_a = find_formants('a.wav')
formants_i = find_formants('i.wav')
formants_bark = find_formants('bark.wav')

print(f"Formants for 'A': {formants_a}")
print(f"Formants for 'I': {formants_i}")
print(f"Formants for Bark: {formants_bark}")
